## [Computational Social Science] Project 5: Natural Language Processing

In this project, you will use natural language processing techniques to explore a dataset containing tweets from members of the 116th United States Congress that met from January 3, 2019 to January 2, 2021. The dataset has also been cleaned to contain information about each legislator. Concretely, you will do the following:

* Preprocess the text of legislators' tweets
* Conduct Exploratory Data Analysis of the text
* Use sentiment analysis to explore differences between legislators' tweets
* Featurize text with manual feature engineering, frequency-based, and vector-based techniques
* Predict legislators' political parties and whether they are a Senator or Representative

You will explore two questions that relate to two central findings in political science and examine how they relate to the text of legislators' tweets. First, political scientists have argued that U.S. politics is currently highly polarized relative to other periods in American history, but also that the polarization is asymmetric. Historically, there were several conservative Democrats (i.e. "blue dog Democrats") and liberal Republicans (i.e. "Rockefeller Republicans"), as measured by popular measurement tools like [DW-NOMINATE](https://en.wikipedia.org/wiki/NOMINATE_(scaling_method)#:~:text=DW\%2DNOMINATE\%20scores\%20have\%20been,in\%20the\%20liberal\%2Dconservative\%20scale.). However, in the last few years, there are few if any examples of any Democrat in Congress being further to the right than any Republican and vice versa. At the same time, scholars have argued that this polarization is mostly a function of the Republican party moving further right than the Democratic party has moved left. **Does this sort of asymmetric polarization show up in how politicians communicate to their constituents through tweets?**

Second, the U.S. Congress is a bicameral legislature, and there has long been debate about partisanship in the Senate versus the House. The House of Representatives is apportioned by population and all members serve two year terms. In the Senate, each state receives two Senators and each Senator serves a term of six years. For a variety of reasons (smaller chamber size, more insulation from the voters, rules and norms like the filibuster, etc.), the Senate has been argued to be the "cooling saucer" of Congress in that it is more bipartisan and moderate than the House. **Does the theory that the Senate is more moderate have support in Senators' tweets?**

**Note**: See the project handout for more details on caveats and the data dictionary.

In [1]:
# pandas and numpy
import pandas as pd
import numpy as numpy

# punctuation, stop words and English language model
from string import punctuation
import spacy
from spacy.matcher import Matcher
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm
from collections import Counter
nlp = en_core_web_sm.load()

# textblob
from textblob import TextBlob

# countvectorizer, tfidfvectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# gensim
import gensim
from gensim import models

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# load data 
# ----------
congress_tweets = pd.read_csv("../../116th Congressional Tweets and Demographics.csv")
# fill in this line of code with a sufficient number of tweets, depending on your computational resources
congress_tweets = congress_tweets.sample(2000).reset_index()
congress_tweets.head()

,index,tweet_id,screen_name,datetime,text,name_wikipedia,position,joined_congress_date,birthday,gender,state,district_number,party,trump_2016_state_share,clinton_2016_state_share,obama_2012_state_share,romney_2012_state_share
0,444448,1.217500e+18,Ilhan,2020-01-15T12:47:19-05:00,We spend more on the Pentagon than the next 7...,Ilhan Omar,Rep,3-Jan-19,10/4/1981,F,MN,5,Democrat,"1,322,951","1,367,716","1,546,167","1,320,225"
1,494749,1.088910e+18,RepChrisPappas,2019-01-25T16:28:46-05:00,RT @jdistaso NH all-D cong. delegation on shut...,Chris Pappas,Rep,3-Jan-19,6/4/1980,M,NH,1,Democrat,"345,790","348,526","369,561","329,918"
2,745680,1.181710e+18,RepMarkGreen,2019-10-08T19:18:35-04:00,Running an impeachment inquiry behind closed d...,Mark E. Green,Rep,3-Jan-19,11/8/1964,M,TN,7,Republican,"1,522,925","870,695","960,709","1,462,330"
3,908195,1.245380e+18,SenatorCantwell,2020-04-01T11:44:08-04:00,The waiver would provide needed flexibilities ...,Maria Cantwell,Sen,3-Jan-01,10/13/1958,F,WA,Senate,Democrat,"1,221,747","1,742,718","1,755,396","1,290,670"
4,790297,1.195150e+18,RepBillFlores,2019-11-14T20:33:12-05:00,RT @SaraCarterDC .@marklevinshow:\n\n“The so-c...,Bill Flores,Rep,3-Jan-11,2/25/1954,M,TX,17,Republican,"4,685,047","3,877,868","3,308,124","4,569,843"


## Preprocessing

The first step in working with text data is to preprocess it. Make sure you do the following:

* Remove punctuation and stop words. The `rem_punc_stop()` function we used in lab is provided to you but you should feel free to edit it as necessary for other steps
* Remove tokens that occur frequently in tweets, but may not be helpful for downstream classification. For instance, many tweets contain a flag for retweeting, or share a URL 

As you search online, you might run into solutions that rely on regular expressions. You are free to use these, but you should also be able to preprocess using the techniques we covered in lab. Specifically, we encourage you to use spaCy's token attributes and string methods to do some of this text preprocessing.

In [3]:
#view just the text column
congress_tweets['text'].head()

0    We spend more on the Pentagon  than the next 7...
1    RT @jdistaso NH all-D cong. delegation on shut...
2    Running an impeachment inquiry behind closed d...
3    The waiver would provide needed flexibilities ...
4    RT @SaraCarterDC .@marklevinshow:\n\n“The so-c...
Name: text, dtype: object

In [4]:
# create a function to remove punctuation and stop words 
def rem_punc_stop(text):

    # set objects
    stop_words = STOP_WORDS         # set STOP_WORDS to a new object variable
    punc = set(punctuation)         # convert punctuation to a set
    
    # essentially remove the punctuation - important to remove punctuation first to correctly capture stop words
    punc_free = "".join([ch for ch in text if ch not in punc]) # join new list of characters (ch) in text w/ condition
                                                               # if ch is not in punctuation 
                                                               # "".join() creates a string from the list comprehension

    # apply nlp to punctuation-free object
    doc = nlp(punc_free)
    
    # extract words from processed text 
    spacy_words = [token.text for token in doc]
    
   # Filter out stopwords, URLs, and specific tokens
    clean_text = [word for word in spacy_words if word.lower() not in stop_words and not word.startswith("http") 
                  and word.lower() not in {"rt", "@", "-","\n","\n\n"}]
    
    return clean_text

In [5]:
# apply the rem_punc_stop function
congress_tweets['clean_text'] = congress_tweets['text'].apply(rem_punc_stop)

In [6]:
congress_tweets['clean_text']

0       [spend, Pentagon,  , 7, military, budgets, com...
1       [jdistaso, NH, allD, cong, delegation, shutdow...
2       [Running, impeachment, inquiry, closed, doors,...
3       [waiver, provide, needed, flexibilities, respo...
4       [SaraCarterDC, marklevinshow, “, socalled, whi...
                              ...                        
1995    [Reported, restrictions, Puerto, Rico, disaste...
1996    [Happy, Birthday, friend, colleague, RepMaxine...
1997    [m, encouraged, Saudi, Arabia, decision, allow...
1998    [Great, meeting, morning, Women, Government, R...
1999    [eagankemp, statement, record, WampM, Medicare...
Name: clean_text, Length: 2000, dtype: object

In [7]:
flat_list = [item for sublist in congress_tweets['clean_text'] for item in sublist]

In [8]:
#view dictionary of tokens
Counter(flat_list)

Counter({'spend': 4,
         'Pentagon': 3,
         ' ': 251,
         '7': 11,
         'military': 32,
         'budgets': 3,
         'combined': 1,
         'nt': 68,
         'need': 148,
         '800': 1,
         'bases': 2,
         'world': 35,
         'country': 100,
         'safe': 47,
         'jdistaso': 1,
         'NH': 8,
         'allD': 1,
         'cong': 1,
         'delegation': 9,
         'shutdown': 11,
         ' \n': 6,
         'SenatorShaheen': 2,
         'reckless': 3,
         'amp': 502,
         'completely': 7,
         'unnecessary': 6,
         'SenatorHassan': 1,
         'happened': 6,
         'RepChrisPappas': 3,
         'pointless': 1,
         'exercise': 4,
         'change': 33,
         'thing': 13,
         'RepAnnieKuster': 1,
         'illadvised': 1,
         'nhpolitics': 2,
         'WMUR': 1,
         'Running': 1,
         'impeachment': 38,
         'inquiry': 11,
         'closed': 4,
         'doors': 7,
         'reveals': 

## Exploratory Data Analysis

Use two of the techniques we covered in lab (or other techniques outside of lab!) to explore the text of the tweets. You should construct these visualizations with an eye toward the eventual classification tasks: (1) predicting the legislator's political party based on the text of their tweet, and (2) predicting whether the legislator is a Senator or Representative. As a reminder, in lab we covered word frequencies, word clouds, word/character counts, scattertext, and topic modeling as possible exploration tools. 

### EDA 1

In [ ]:
... 

### EDA 2

In [ ]:
...

## Sentiment Analysis

Next, let's analyze the sentiments contained within the tweets. You may use TextBlob or another library for these tasks. Do the following:

* Choose two legislators, one who you think will be more liberal and one who you think will be more conservative, and analyze their sentiment and/or subjectivity scores per tweet. For instance, you might do two scatterplots that plot each legislator's sentiment against their subjectivity, or two density plots for their sentiments. Do the scores match what you thought?
* Plot two more visualizations like the ones you chose in the first part, but do them to compare (1) Democrats v. Republicans and (2) Senators v. Representatives 

`TextBlob` has already been imported in the top cell.

In [ ]:
...

## Featurization

Before going to classification, explore different featurization techniques. Create three dataframes or arrays to represent your text features, specifically:

* Features engineered from your previous analysis. For example, word counts, sentiment scores, topic model etc.
* A term frequency-inverse document frequency matrix. 
* An embedding-based featurization (like a document averaged word2vec)

In the next section, you will experiment with each of these featurization techniques to see which one produces the best classifications.

In [ ]:
...

### Engineered Text Features

In [ ]:
# Engineered Features
...

### Bag-of-words or Tf-idf

In [ ]:
# Frequency Based featurization
...

### Word Embedding

In [ ]:
# Load Word2Vec model from Google; OPTIONAL depending on your computational resources (the file is ~1 GB)
# Also note that this file path assumes that the word vectors are underneath 'data'; you may wish to point to the CSS course repo and change the path
# or move the vector file to the project repo 

#model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary = True) 

In [ ]:
# Function to average word embeddings for a document; use examples from lab to apply this function. You can use also other techniques such as PCA and doc2vec instead.
def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in model.vocab]
    return np.mean(model[doc], axis=0)

In [ ]:
# embedding based featurization
...

## Classification

Either use cross-validation or partition your data with training/validation/test sets for this section. Do the following:

* Choose a supervised learning algorithm such as logistic regression, random forest etc. 
* Train six models. For each of the three dataframes you created in the featurization part, train one model to predict whether the author of the tweet is a Democrat or Republican, and a second model to predict whether the author is a Senator or Representative.
* Report the accuracy and other relevant metrics for each of these six models.
* Choose the featurization technique associated with your best model. Combine those text features with non-text features. Train two more models: (1) A supervised learning algorithm that uses just the non-text features and (2) a supervised learning algorithm that combines text and non-text features. Report accuracy and other relevant metrics. 

If time permits, you are encouraged to use hyperparameter tuning or AutoML techniques like TPOT, but are not explicitly required to do so.

### Train Six Models with Just Text

In [ ]:
# six models ([engineered features, frequency-based, embedding] * [democrat/republican, senator/representative])
...

### Two Combined Models

In [ ]:
# two models ([best text features + non-text features] * [democrat/republican, senator/representative])
...

## Discussion Questions

1. Why do standard preprocessing techniques need to be further customized to a particular corpus?

**YOUR ANSWER HERE** ...

2. Did you find evidence for the idea that Democrats and Republicans have different sentiments in their tweets? What about Senators and Representatives?

**YOUR ANSWER HERE** ...

3. Why is validating your exploratory and unsupervised learning approaches with a supervised learning algorithm valuable?

**YOUR ANSWER HERE** ...

4. Did text only, non-text only, or text and non-text features together perform the best? What is the intuition behind combining text and non-text features in a supervised learning algorithm?

**YOUR ANSWER HERE** ...